In [11]:
import pandas as pd
import re
import nltk
import time
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ACER\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ACER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [12]:
#Step 1-Load dataset and  cleaning iyup
df = pd.read_csv("Rawdata_1000.csv")
df = df[['Judul', 'Content']]
df.to_csv('step1_cleaning_result.csv', index=False)

In [13]:
#Step1-Filtering
def filtering_content(text):
    if not isinstance(text, str):
        return ""
    
    # 1. Standarisasi karakter (hapus non-breaking space)
    text = text.replace('\xa0', ' ')
    
    # 2. Hapus Iklan & Navigasi Spesifik
    junk_phrases = [
        "Baca berita dengan sedikit iklan,  klik di sini",
        "Baca berita dengan sedikit iklan, klik di sini",
        "Scroll ke bawah untuk melanjutkan membaca",
        "Ringkasan Berita",
        "INFO NASIONAL –",
        "INFO TEMPO –",
        "(*) "
    ]
    for phrase in junk_phrases:
        text = text.replace(phrase, " ")

    # 3. Hapus Header Media di awal teks (Lebih Fleksibel)
    # Menangani: "TEMPO.CO, Jakarta -", "Solo -", "JAKARTA, KOMPAS.com -"
    # Menggunakan \s* di depan untuk antisipasi spasi liar di awal
    text = re.sub(r'^\s*[A-Za-z0-9\.]+\s*,\s*[A-Za-z\s]+\s*[-–—]\s*', '', text) 
    text = re.sub(r'^\s*[A-Z\s]+,\s*[A-Za-z0-9\.]+\s*[-–—]\s*', '', text)
    text = re.sub(r'^\s*[A-Z][a-z]+\s*[-–—]\s*', '', text) # Contoh: Solo -
    text = re.sub(r'^\s*[A-Z\s]+\s*[-–—]\s*', '', text)    # Contoh: JAKARTA -

    # 4. Hapus Link Video & Pilihan Editor (beserta judul beritanya)
    # Menghapus frasa dan teks setelahnya sampai ketemu baris baru atau spasi ganda
    text = re.sub(r'(Simak|Lihat|Baca)\s+Juga\s+Video:?.*?(?=\n|\r|\s{2,}|$)', ' ', text, flags=re.IGNORECASE)
    text = re.sub(r'Pilihan\s+Editor:?.*?(?=\n|\r|\s{2,}|$)', ' ', text, flags=re.IGNORECASE)
    
    # 5. Hapus Kredit Metadata (Editor, Penulis, Foto, dll)
    text = re.sub(r'(Video|Editor|Sumber|Foto|Videographer|Penulis|Reporter):\s*.*?(?=\n|\r|\s{2,}|$)', ' ', text, flags=re.IGNORECASE)
    
    # 6. Hapus Atribusi Foto/Kontributor
    text = re.sub(r'Dok\.\s+[A-Za-z\s]+', ' ', text)
    text = re.sub(r'[A-Z\s|]+\sberkontribusi dalam penulisan artikel ini\.', ' ', text)

    # 7. Bersihkan spasi berlebih
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Terapkan pembersihan
df['Content'] = df['Content'].apply(filtering_content)

# Simpan hasil
df.to_csv('step1_filtering_result.csv', index=False)
print("Step1_Filtering_result.csv")
print(df['Content'].head())

Step1_Filtering_result.csv
0    Wali Kota Malang, Wahyu Hidayat, memberikan du...
1    Presiden Prabowo Subianto mengungkapkan rencan...
2    Warga Jalan Eretan 2, Condet, Jakarta Timur, m...
3    Bareskrim Polri belum menetapkan tersangka dal...
4    Masih ingat bentrokan di Belanda pada November...
Name: Content, dtype: str


In [14]:
#Step 1 - merged
df['Merged'] = df['Judul'] + " " + df['Content']
df[['Merged']].to_csv('step1_merge_result.csv', index=False)

In [15]:
#Step 2 - Case folding
df['Merged'] = df['Merged'].str.lower()
df[['Merged']].to_csv('step2_casefolding_result.csv', index=False)


In [16]:
#Step 3 - Punctuatiation removal
def remove_punctuation(text):
    return re.sub(r'[^a-z\s]', ' ', text)

df['Merged'] = df['Merged'].apply(remove_punctuation)
df[['Merged']].to_csv('step3_punctuation_result.csv', index=False)



In [17]:
#Step 4 - Tokenization
def tokenizing(text):
    return text.split()

df['Merged'] = df['Merged'].apply(tokenizing)

df.to_csv('step4_tokenization_result.csv', index=False)

print("hasil Step Tokenization")
print(df['Merged'].head())

hasil Step Tokenization
0    [wali, kota, malang, beri, dukungan, dan, moti...
1    [prabowo, beberkan, rencana, pengembangan, kom...
2    [curhat, ke, rano, karno, warga, condet, selam...
3    [asas, praduga, tak, bersalah, polri, belum, t...
4    [sidang, kasus, bentrok, amsterdam, orang, dih...
Name: Merged, dtype: object


In [ ]:
#step5 - Normalisasi 
kamus_normalisasi = {
    # --- Institusi Pemerintahan, Hukum & Keamanan ---
    'kejagung': 'kejaksaan agung',
    'bareskrim': 'badan reserse kriminal',
    'pn': 'pengadilan negeri',
    'dpd': 'dewan perwakilan daerah',
    'dpr': 'dewan perwakilan rakyat',
    'dprd': 'dewan perwakilan rakyat daerah',
    'mpr': 'majelis permusyawaratan rakyat',
    'mk': 'mahkamah konstitusi',
    'ma': 'mahkamah agung',
    'kpk': 'komisi pemberantasan korupsi',
    'kpu': 'komisi pemilihan umum',
    'bawaslu': 'badan pengawas pemilihan umum',
    'bpbd': 'badan penanggulangan bencana daerah',
    'bpkp': 'badan pengawasan keuangan dan pembangunan',
    'bmkg': 'badan meteorologi klimatologi dan geofisika',
    'bin': 'badan intelijen negara',
    'mabes': 'markas besar',
    'pbb': 'perserikatan bangsa bangsa',
    'pemkot': 'pemerintah kota',
    'pemprov': 'pemerintah provinsi',
    'pemkab': 'pemerintah kabupaten',
    'pemda': 'pemerintah daerah',
    'kadinsos': 'kepala dinas sosial',
    'kadinkes': 'kepala dinas kesehatan',
    'kabag': 'kepala bagian',

    # --- Kepolisian & Militer ---
    'tni': 'tentara nasional indonesia',
    'polri': 'kepolisian negara republik indonesia',
    'polda': 'kepolisian daerah',
    'polres': 'kepolisian resor',
    'polsek': 'kepolisian sektor',
    'akbp': 'ajun komisaris besar polisi',
    'akp': 'ajun komisaris polisi',
    'tpnpb': 'tentara pembebasan nasional papua barat',

    # --- Kementerian & Pejabat ---
    'kemenag': 'kementerian agama',
    'kemendikbud': 'kementerian pendidikan dan kebudayaan',
    'kemenkes': 'kementerian kesehatan',
    'kemenkeu': 'kementerian keuangan',
    'kemenhub': 'kementerian perhubungan',
    'kemenlu': 'kementerian luar negeri',
    'kemhan': 'kementerian pertahanan',
    'menko': 'menteri koordinator',
    'mendagri': 'menteri dalam negeri',
    'pj': 'penjabat',
    'plt': 'pelaksana tugas',
    'pm': 'perdana menteri',
    'eks': 'mantan',
  

    # --- Partai Politik, Koalisi & Pemilu ---
    'pdip': 'partai demokrasi indonesia perjuangan',
    'pdi': 'partai demokrasi indonesia',
    'dpp': 'dewan pimpinan pusat',
    'pks': 'partai keadilan sejahtera',
    'pkb': 'partai kebangkitan bangsa',
    'ppp': 'partai persatuan pembangunan',
    'gol': 'golongan',
    'kim': 'koalisi indonesia maju',
    'munas': 'musyawarah nasional',
    'paslon': 'pasangan calon',
    'pilkada': 'pemilihan kepala daerah',
    'pemilu': 'pemilihan umum',
    'psu': 'pemungutan suara ulang',
    'cawagub': 'calon wakil gubernur',
    'cagub': 'calon gubernur',
    'cabup': 'calon bupati',
    'cawabup': 'calon wakil bupati',
    'tps': 'tempat pemungutan suara',

    # --- Istilah Birokrasi, Hukum & Tata Kota ---
    'kuhp': 'kitab undang undang hukum pidana',
    'ham': 'hak asasi manusia',
    'rt': 'rukun tetangga',
    'rw': 'rukun warga',
    'pp': 'peraturan pemerintah',
    'sk': 'surat keputusan',
    'uu': 'undang undang',
    'uud': 'undang undang dasar',
    'ruu': 'rancangan undang undang',
    'perpres': 'peraturan presiden',
    'asn': 'aparatur sipil negara',
    'pns': 'pegawai negeri sipil',
    'jl': 'jalan',

    # --- Istilah Umum, Ekonomi & Bisnis ---
    'umkm': 'usaha mikro kecil dan menengah',
    'bbm': 'bahan bakar minyak',
    'ppn': 'pajak pertambahan nilai',
    'psn': 'proyek strategis nasional',
    'tkp': 'tempat kejadian perkara',
    'thr': 'tunjangan hari raya',
    'hut': 'hari ulang tahun',
    'csr': 'corporate social responsibility',
    'tbk': 'terbuka',
    'pln': 'perusahaan listrik negara',
    'bumn': 'badan usaha milik negara',
    'bca': 'bank central asia',
    'pt': 'perseroan terbatas',
    'rp': 'rupiah',
    'sdm': 'sumber daya manusia',

    # --- Spesifik Konteks Berita, Tokoh & Ukuran ---
    'mbg': 'makan bergizi gratis',
    'gbk': 'gelora bung karno',
    'ikn': 'ibu kota nusantara',
    'dki': 'daerah khusus ibukota',
    'hp': 'handphone',
    'dr': 'dokter',
    'sby': 'susilo bambang yudhoyono',
    'ahy': 'agus harimurti yudhoyono',
    'rk': 'ridwan kamil',

    # --- Pendidikan, Kesehatan & Fasilitas ---
    'rsud': 'rumah sakit umum daerah',
    'rs': 'rumah sakit',
    'ponpes': 'pondok pesantren',
    'mts': 'madrasah tsanawiyah',
    'sd': 'sekolah dasar',
    'smp': 'sekolah menengah pertama',
    'sma': 'sekolah menengah atas',
    'smk': 'sekolah menengah kejuruan',

    # --- Kewarganegaraan & Kata Hubung/Chat ---
    'wn': 'warga negara',
    'wna': 'warga negara asing',
    'wni': 'warga negara indonesia',
    'ri': 'republik indonesia',
    'wib': 'waktu indonesia barat',
    'yg': 'yang',
    'dgn': 'dengan'
}

def normalize_tokens(tokens):
    return [kamus_normalisasi.get(word, word) for word in tokens]

df['Merged'] = df['Merged'].apply(normalize_tokens)

# Gabungkan kembali jadi string final
df['Final_Text'] = df['Merged'].apply(lambda x: ' '.join(x))

df[['Final_Text']].to_csv('step5_normalization_result.csv', index=False)

print("Haa sudah jugakkkkk")
print(df['Final_Text'].head())

Haa sudah jugakkkkk
0    wali kota malang beri dukungan dan motivasi ke...
1    prabowo beberkan rencana pengembangan komodita...
2    curhat ke rano karno warga condet selama ini k...
3    asas praduga tak bersalah kepolisian negara re...
4    sidang kasus bentrok amsterdam orang dihukum k...
Name: Final_Text, dtype: str


In [19]:
#Step 6 - stopword
factory = StopWordRemoverFactory()
sastrawi_stopwords = factory.get_stop_words()

indonesian_stopwords = stopwords.words('indonesian')
english_stopwords = stopwords.words('english')

all_stopwords = set(sastrawi_stopwords + indonesian_stopwords + english_stopwords)

def remove_stopwords(tokens):
    return [word for word in tokens if word not in all_stopwords]

df['Merged'] = df['Merged'].apply(remove_stopwords)
df[['Merged']].to_csv('step6_stopword_result.csv', index=False)



In [20]:
#Step 7 - Stemming
print("Memulai proses stemming...")
start_time = time.time()

factory = StemmerFactory()
stemmer = factory.create_stemmer()

term_dict = {}

def stemming_with_cache(word):
    if word not in term_dict:
        term_dict[word] = stemmer.stem(word)
    return term_dict[word]

def stemming(tokens):
    return [stemming_with_cache(word) for word in tokens]

df['Merged'] = df['Merged'].apply(stemming)

end_time = time.time() 
print(f"Selesai stemming dalam {(end_time - start_time)/60:.2f} menit")

df[['Merged']].to_csv('step7_stemming_result.csv', index=False)


Memulai proses stemming...
Selesai stemming dalam 10.99 menit
